## Assignment 4B
The goal of this assignment is to set up all relevant code snippets for solving ODEs and calculating roots in a modular form, in preparation for the topic of next week, where we will solve the Schrodinger equation, using all of these components, simultaneously.

For this purpose, you will implement numerically the illustrative example of the shooting method outlined in class, and compare your numerical results with the exact solution.

### Problem 1 (100 points)
Implement the shooting method to solve a boundary value problem using both the bisection and secant methods for root finding. The differential equation describing the motion is:
$$\begin{align}
\dfrac{d^2y}{dt^2}&=-g
\end{align}$$
with initial conditions:
$$\begin{align}
y(0)=0\\
\left.\dfrac{dy}{dx}\right\lvert_{t=0}=v_0
\end{align}$$
The objective is to find the value of $v_0$ such that $y(T) = 0$ for $T = 10$ seconds.

In this exercise, you are explicitly asked to use the 4th-order Runge–Kutta (RK4) method with the same input-output structure as provided in solutions to last week’s assignments (of course, up to convention preferences such as functions and variables names or order).

#### Problem 1.A (30 points)
Implement the bisection method using the input-output structure outlined in Lecture 4 (up to convention preferences such as functions and variables names or order):

In [1]:
def continuous_binary_search_value(*,
    f: callable, 
    lowerbound: int | float, 
    upperbound: int | float, 
    error: int | float, 
    max_iter: int
) -> float:
    """
    Binary Search Method for Continuous Functions (Single Variable)
    Returns the value of x that is the root of the function

    Parameters
    ----------
    f : callable
        The function to find the root of
    lowerbound : int | float
        The lower bound of the search
    upperbound : int | float
        The upper bound of the search
    error : int | float
        The error tolerance
    max_iter : int
        The maximum number of iterations

    Returns
    -------
    float
        The value of x that is the root of the function
    """

    # Type Checking
    assert callable(f)
    assert isinstance(lowerbound, (int, float))
    assert isinstance(upperbound, (int, float))
    assert lowerbound < upperbound
    assert isinstance(error, (int, float))
    assert error > 0
    assert isinstance(max_iter, int)
    assert max_iter > 0

    # Implementation
    for _ in range(max_iter):
        midpoint = (lowerbound + upperbound) / 2
        if f(midpoint) == 0 or (upperbound - lowerbound) / 2 < error:
            print(f"(Binary) Iterations: {_+1}")
            return midpoint
        elif f(midpoint) < 0:
            lowerbound = midpoint
        else:
            upperbound = midpoint
    else:
        print(f"(Binary) Iterations: {max_iter}")
        return midpoint


#### Problem 1.B (30 points)
Implement the secant method using the input-output structure outlined in Lecture 4 (up to convention preferences such as functions and variables names or order):

In [2]:
import numpy as np
def continuous_secant_method_value(*,
    f: callable, 
    x0: int | float, 
    x1: int | float, 
    error: int | float, 
    max_iter: int
) -> float:
    """
    Secant Method for Continuous Functions (Single Variable)
    Returns the value of x that is the root of the function

    Parameters
    ----------
    f : callable
        The function to find the root of
    x0 : int | float
        The first guess of the root
    x1 : int | float
        The second guess of the root
    error : int | float
        The error tolerance
    max_iter : int
        The maximum number of iterations

    Returns
    -------
    float
        The value of x that is the root of the function
    """

    # Type Checking
    assert callable(f)
    assert isinstance(x0, (int, float))
    assert isinstance(x1, (int, float))
    assert x0 != x1, "Guesses x0 and x1 cannot be the same"
    assert isinstance(error, (int, float))
    assert error > 0
    assert isinstance(max_iter, int)
    assert max_iter > 0

    # Implementation
    for _ in range(max_iter):
        x2 = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        if np.abs(x2 - x1) < error:
            print(f"(Secant) Iterations: {_+1}")
            return x2
        x0, x1 = x1, x2
    else:
        print(f"(Secant) Iterations: {max_iter}")
        return x2


#### Problem 1.C (40 points)
Use your RK4 implementation as a subroutine in combination with both the bisection and secant methods to find the initial velocity $v_0$ such that $y(T) = 0$. Use $g = 9.8 \text{ m/s}^2$ and a tolerance of $10^{-6}$ for the root-finding methods.
- Report the values of v0 obtained with each method and compare to the exact solution.
- Why does the secant method converge so fast?

In [3]:
import numpy as np
def continuous_runge_kutta_method_value(
    f: callable,
    t0: float | int,
    x0: float | int | np.ndarray | tuple | list,
    dt: float | int,
    tf: float | int
) -> float:
    """
    Runge-Kutta Method for solving ordinary differential equations.
    This function solves the ODE using the Runge-Kutta Method and returns the final value.
    
    Parameters:
        f: callable - The function equal to the derivative of the unknown function
        t0: float | int - The initial time
        x0: float | int - The initial value(s)
        dt: float | int - The time step
        tf: float | int - The final time
    
    Returns:
        float - The value of the unknown function at the final time
    """
    
    # Input Checking
    assert callable(f)
    assert isinstance(t0, (int, float))
    assert isinstance(x0, (int, float, np.ndarray, tuple, list))
    assert isinstance(dt, (int, float))
    assert isinstance(tf, (int, float))

    # Implementation
    T = np.arange(t0, tf+dt, dt)
    x = x0
    for t in T[:-1]:
        k1 = f(t       , x          )
        k2 = f(t + dt/2, x + k1*dt/2)
        k3 = f(t + dt/2, x + k2*dt/2)
        k4 = f(t + dt  , x + k3*dt  )
        x += (k1 + 2*k2 + 2*k3 + k4)*dt/6
    return x


# Binary Search Method
MAX_ITERATIONS = 1000
v1, v2 = 0.0, 1000.0
starting_velocity = continuous_binary_search_value(
    f=lambda v: continuous_runge_kutta_method_value(
        f=lambda t, x: np.array([x[1], -9.81]), 
        t0=0, x0=np.array([0, v]), dt=0.01, tf=10
    )[0],
    lowerbound=v1, upperbound=v2, error=1e-6, max_iter=MAX_ITERATIONS
)
ground_velocity = continuous_runge_kutta_method_value(
    f=lambda t, x: np.array([x[1], -9.81]),
    t0=0, x0=np.array([0, starting_velocity]), dt=0.001, tf=10
)[1]
print(f"(Binary) The velocity when the projectile hits the ground is {ground_velocity:.3f} m/s")


# Secant Method
MAX_ITERATIONS = 1000
ERROR = 1e-6
v0, v1 = 25.0, 100.0
F: lambda t, x: np.array([x[1], -9.81])
starting_velocity = continuous_secant_method_value(
    f=lambda v: continuous_runge_kutta_method_value(
        f=lambda t, x: np.array([x[1], -9.81]),
        t0=0, x0=np.array([0, v]), dt=0.01, tf=10
    )[0],
    x0=v0, x1=v1, error=ERROR, max_iter=MAX_ITERATIONS
)
ground_velocity = continuous_runge_kutta_method_value(
    f=lambda t, x: np.array([x[1], -9.81]), 
    t0=0, x0=np.array([0, starting_velocity]), dt=0.001, tf=10
)[1]
print(f"(Secant) The velocity when the projectile hits the ground is {ground_velocity:.3f} m/s")

(Binary) Iterations: 30
(Binary) The velocity when the projectile hits the ground is -49.050 m/s
(Secant) Iterations: 2
(Secant) The velocity when the projectile hits the ground is -49.050 m/s


Both of the systems get our value of $v=-49.050$. The exact answer is $v=-(9.81*10/2)=49.05$ using projectile motion kinematics.

The secant method converges as fast as it does because of the geometry of the system.

Since it requires 2 points, it converges to the zero very quickly since it is locally linear about the point of the zero.